In [ ]:
from gensim.models.coherencemodel import CoherenceModel
import pandas as pd
import time
import gc
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install tomotopy 

In [ ]:
!pip install numpy --upgrade --user

In [ ]:
import tomotopy as tp

In [ ]:
df = pd.read_csv('/content/drive/My Drive/datasets/articles/monthPre/final/completo/preprocessedCompletoComBigrams.csv')
ldamodel = tp.LDAModel
TermWeight = tp.TermWeight

In [ ]:
ldamodel = tp.LDAModel
TermWeight = tp.TermWeight

In [ ]:
co = tp.coherence

In [ ]:
def topicos_sem_prob(LDA):
  tp_lda = []
  for k in range(LDA.k):
    aux = []
    lista = LDA.get_topic_words(k, top_n=20)
    tp_lda.append([t[0] for t in lista])
  topics = [] 
  for doc in tp_lda:
    topics.append(", ".join(word for word in doc))
  return topics

In [ ]:

#coherenceDict = {'month': [], 'topic': [], 'iter': [], 'c_v':[], 'alpha': [], 'eta':[], 'mindf': [], 'mincf':[], 'rmtop': [], 'vocab': []}
topics = [ 30, 35, 40, 45] #30,35,40 
alphas = [0.1, 0.05, 0.01 ]
etas = [ 0.1, 0.05, 0.01]
rmtop =[50, 100, 200]
mindf = [100, 200, 400] #min document frequency 
mincf = [200, 300, 400, 500]
iter = 500
months = [6] #1 ja foi
for month in months:
  bestCV = 0
  coherenceDict = {'month': [], 'topic': [], 'iter': [], 'c_v':[], 'alpha': [], 'eta':[], 'mindf': [], 'mincf':[], 'rmtop': [], 'vocab': []}
  init_month = time.perf_counter() 
  print("\n\n ---------------- MES {} ------------------------ \n\n".format(month))
  print("Quantidade de noticias nesse mes: {}".format(df.content[df['month'] == month].shape[0]))
  for t in topics:
    for mcf in mincf:
      for rm in rmtop:
        for a in alphas:
          for e in etas:
            for mdf in mindf:
              
              gc.collect()
              tic = time.perf_counter() ## time
            
              #corpus = tp.utils.Corpus(tokenizer=tp.utils.SimpleTokenizer())
              #corpus.process(df.content[df['month'] == month])   
              #LDA = ldamodel(corpus = corpus, tw= TermWeight.IDF, k= t, alpha= a, eta= e, seed= 9999, min_df =mdf, min_cf = mcf, rm_top=rm)
              print("Gerando modelo...")
              LDA = ldamodel(tw= TermWeight.IDF, k= t, alpha= a, eta= e, seed= 9999, min_df =mdf, min_cf = mcf, rm_top=rm)
              qntd = 0
              for doc in df.content[df['month'] == month]:
                qntd+=1
                LDA.add_doc(doc.split(' '))
              print("docs", qntd)
              print("Treinando...")
              LDA.train(iter = iter)
              print("Coherence...")
              coh = co.Coherence(LDA, coherence= 'c_v', top_n=10 )
              valuecv = coh.get_score()
              print('Num docs:', len(LDA.docs), ' Num words', LDA.num_words, 'num vocabs', LDA.num_vocabs)
              print("topic {} iter {} c_v {} alpha {} eta {} rm {} mindf {} mincf {} month {}".format(t, iter, valuecv, a, e, rm, mdf, mcf, month))

              if valuecv > bestCV:
                bestCV = valuecv
                print("Salvando melhor modelo para o mês {}...".format(month))

                init_t = time.perf_counter() ## time
                LDA.save('/content/drive/My Drive/datasets/articles/monthPre/final/parametros/melhormodelo/melhorModeloMes{}C.bin'.format(month), full = True)
                end_t = time.perf_counter()
                print("Tempo para salvar o modelo {}s".format(round((end_t - init_t), 2)))

                print("Salvando melhores topicos do mes {}".format(month))
                topicosCsv = pd.DataFrame();
                topicosStr = topicos_sem_prob(LDA)
                topicosCsv['topico'] = topicosStr
                topicosCsv.to_csv('/content/drive/My Drive/datasets/articles/monthPre/final/parametros/topicosmeses/melhoresTopicosMes{}C.csv'.format(month))

              toc = time.perf_counter() ## time
              print("Tempo de execução final {}s\n".format(round((toc-tic), 2)))

              coherenceDict['topic'].append(t)
              coherenceDict['month'].append(month)
              coherenceDict['iter'].append(iter)
              coherenceDict['c_v'].append(valuecv)
              coherenceDict['alpha'].append(a)
              coherenceDict['eta'].append(e)
              coherenceDict['rmtop'].append(rm)
              coherenceDict['mindf'].append(mdf)
              coherenceDict['mincf'].append(mcf)
              coherenceDict['vocab'].append(LDA.num_vocabs)
              gc.collect()
              corpus = None
              LDA = None
              dfDados = pd.DataFrame.from_dict(coherenceDict)
              #dfDados.to_csv('/content/drive/My Drive/datasets/articles/monthPre/final/parametros/dados/dadosTopicosMes{}C.csv'.format(month))
              
  end_month = time.perf_counter()
  print("\n--- TEMPO DE EXECUÇÃO DO MÊS {}: {}s\n".format(month, round((end_month - init_month), 2)))

params escolhidos para cada mes:

alpha: 0.01, eta: 0.05, topicos: 30, min_df = 400, rm_top: 100, min_cf = 300
